In [1]:
import os

Needed to construct database for RAG

In [2]:
from langchain_community.vectorstores import FAISS  # "db" to store and retrieve embeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter  # split long documents
from pdfminer.high_level import extract_text  # extract text from pdfs
from langchain_huggingface import HuggingFaceEmbeddings
from pathlib import Path
import logging

In [3]:
dir_path = Path("./scraped_pdfs")

In [4]:
# Silence pdfminer logs below ERROR
logging.getLogger("pdfminer").setLevel(logging.ERROR)

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
doc_lst = []
# Extract text
#text = extract_text(DOC_PATH, page_numbers=[0])
for i, file in enumerate(dir_path.iterdir()):
    # if i >= 20:
    #     break

    if file.name.endswith(".pdf") and not file.name.startswith("slvfs"):
        text = extract_text(file)

        doc = Document(page_content=text, metadata={"source": file.name})
        chunks = splitter.split_documents([doc])
        doc_lst.extend(chunks) # avoid nested lists

    else:
        print(f"Skipping {file.name}")

Skipping slvfs-2000-46-kons.pdf
Skipping slvfs-1998-8.pdf
Skipping slvfs-2000-22.pdf
Skipping slvfs-2000-16-andr-1997-27.pdf
Skipping slvfs-1998-38-andr-1997-27.pdf
Skipping slvfs-1980-06-kons.pdf
Skipping slvfs-1995-17.pdf
Skipping slvfs-1995-19-andr-1980-6-inkl-bil3-skannad.pdf
Skipping slvfs-2000-29-andr-1997-27.pdf
Skipping slvfs-2000-46.pdf
Skipping slvfs-1998-08-kons.pdf
Skipping slvfs-1997-27-kons-2016-9_2.pdf
Skipping slvfs-1995-18-bil-1-3.pdf
Skipping slvfs-1999-24-andr-1998-8.pdf
Skipping slvfs-1997-27.pdf
Skipping slvfs-1980-04-kons.pdf
Skipping slvfs-1980-05-kons.pdf
Skipping slvfs-1993-20-kons.pdf
Skipping slvfs-1998-21-andr-1998-8.pdf
Skipping slvfs-1995-18.pdf
Skipping slvfs-1993-20.pdf
Skipping slvfs-1980-5_inskannad.pdf
Skipping slvfs-1080-4-inskannad.pdf
Skipping slvfs-1999-5-andr-1998-8.pdf
Skipping slvfs-1980-6_inskannad.pdf


In [6]:
print(len(doc_lst)) # number of chunks

195876


In [7]:
# Local embedding model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [8]:
# Database
db = FAISS.from_documents(doc_lst, embeddings)

In [9]:
# Save the database
db.save_local("faiss_index_all")

In [10]:
# Given a query, retrieve the most relevant chunk(s) and appropriately prompt your LLM

query = "Bisfenol A"
results = db.similarity_search(query, k=10)

for r in results:
    print(r.page_content)

”6. 
Genom  undantag  från  artikel  5  får  2,2-bis(4-hydroxifenyl)propan  (bisfenol  A)  (CAS-nr  80-05-7)  och  andra
A) 

Struktur för bisfenol

B) 

Struktur för bisfenolderivat
bisfenol-A-diglycidyleter  (CAS-nr  1675-54-3),  och  följaktligen  utgöra  en  del  av  dessa  ämnens  kemiska  struktur,
innehåller bisfenol A, en annan farlig bisfenol eller ett farligt bisfenolderivat, eller  inte avger dessa ämnen i livsmedel över
Ämnet  2,2-bis(4-hydroxifenyl)propan 
(CAS-nr  0000080-05-7),  vanligen  kallat  bisfenol  A,  används  vid
bisfenol: ämne som består av två hydroxifenylgrupper som är sammankopplade med en atom, i enlighet med struktur
Godkännande för användning av andra farliga bisfenoler än bisfenol A eller av farliga bisfenolderivat vid
Förbud mot användning av andra farliga bisfenoler än bisfenol A eller av farliga bisfenolderivat

Artikel 5
2,4-D

2-Fenylfenol

Abamektin

Acefat

Acetamiprid

Aklonifen

Akrinatrin

Aldikarb

Aldrin och dieldrin

Ametoktradin

Azinfosme